In [2]:
from trainer import CatDogTrainer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1 - original training

Here we train the original model from [here](https://www.tensorflow.org/tutorials/images/transfer_learning). We get very good accuracy on both train and validation sets (more than .9) after just 2-3 epochs.

In [3]:
trainer = CatDogTrainer(initial_epochs=3, model_type='mobile_net')

In [4]:
trainer.train()

Epoch 1/3
582/582 [==============================] - 77s 132ms/step - loss: 2.5899 - accuracy: 0.6847 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 39s 66ms/step - loss: 1.4116 - accuracy: 0.8271 - val_loss: 0.9865 - val_accuracy: 0.8871
Epoch 3/3
582/582 [==============================] - 39s 67ms/step - loss: 0.9741 - accuracy: 0.8763 - val_loss: 0.7353 - val_accuracy: 0.9181


# 2 - resnet

Let's now change `MobileNetV2` to `resnet50` without changing anything else. Result are not bad for validation set but for some reason train accuracy is below than validation accuracy.

In [5]:
trainer = CatDogTrainer(initial_epochs=3, model_type='resnet50')

94674944/94668760 [==============================] - 1s 0us/step


In [6]:
trainer.train()

Epoch 1/3
582/582 [==============================] - 59s 102ms/step - loss: 2.4497 - accuracy: 0.6938 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 53s 91ms/step - loss: 1.3165 - accuracy: 0.8233 - val_loss: 0.6755 - val_accuracy: 0.9427
Epoch 3/3
582/582 [==============================] - 53s 91ms/step - loss: 1.0544 - accuracy: 0.8525 - val_loss: 0.5614 - val_accuracy: 0.9509
